## 数据检索
用户输入只是问题，答案需要从已知的数据库或者文档中获取。因此，我们需要使用检索器获取上下文，并通过“question”键下的用户输入。

### RAG
检索增强生成（Retrieval-augmented Generation，RAG），是当下最热门的大模型前沿技术之一。如果将“微调（finetune）”理解成大模型内化吸收知识的过程，那么RAG就相当于给大模型装上了“知识外挂”，基础大模型不用再训练即可随时调用特定领域知识。

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings_path = "/Users/libing/kk_LLMs/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)



/var/folders/f5/wwjgctq529v381m5rw2lrfr00000gn/T/ipykernel_99416/3003229924.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)
/Users/libing/miniconda3/envs/kk_agent/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
vectorstore = FAISS.from_texts(["小明在华为工作", "熊喜欢吃蜂蜜"], 
                               embeddings)

vectorstore

In [5]:
# 使用向量数据库生成检索器
retriever = vectorstore.as_retriever()

# 使用检索器获取上下文
context = retriever.invoke("小明喜欢吃什么")

context


[Document(metadata={}, page_content='小明在华为工作'),
 Document(metadata={}, page_content='熊喜欢吃蜂蜜')]

In [6]:
# 连接本地大模型
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3)


In [7]:
from langchain.prompts import ChatPromptTemplate

template = """
你是一个调查员，请根据以下信息回答问题：
{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [8]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

setup_and_retriever = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
)

chain = setup_and_retriever | prompt | chat | output_parser

chain.invoke("小明喜欢吃什么")


'根据提供的信息，我们无法得知小明的具体饮食偏好或他是否喜欢吃某种特定的食物。给出的信息中只提到“小明在华为工作”，并没有关于他的饮食习惯的描述。因此，对于“小明喜欢吃什么”的问题，目前没有足够的信息来回答。如果需要了解小明的喜好，可能需要更多的相关信息或者直接询问小明本人。'

In [9]:
#完整链式
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

chain.invoke("小明在哪里工作？")

'小明在华为工作。'

In [16]:
# 一个完整的示例
from operator import itemgetter

template = """
你是一个调查员，请根据以下信息回答问题：
{context}

问题：{question}
回答问题时请礼貌称呼: {name}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "name": itemgetter("name")
    }
    | prompt
    | chat
    | output_parser
)

for m in chain.stream({"question": "熊喜欢吃什么", "name": "调查员"}):
    print(m, end="", flush=True)


您好，根据提供的信息，调查员可以告诉您，熊喜欢吃蜂蜜。